# Kili Tutorial: Importing inference labels

In this tutorial, we will walk through the process of using Kili to evaluate the performance of a machine learning model in production. The goal of this tutorial is to illustrate how to push such labels, and how to visualize the quality of those predicted labels.

Additionally:

For an overview of Kili, visit https://kili-technology.com. You can also check out the Kili documentation https://cloud.kili-technology.com/docs.

The tutorial is divided into two parts:

1. Giving a bit of context
2. How to make use of inference labels in practice

This next cell connects the notebook to the Kili API. You need to update the credentials `api_key` before.

In [1]:
import os
#!pip install kili
from kili.client import Kili

api_key = os.getenv('KILI_USER_API_KEY')
api_endpoint = os.getenv('KILI_API_ENDPOINT')
# If you use Kili SaaS, use the url 'https://cloud.kili-technology.com/api/label/v2/graphql'

kili = Kili(api_key=api_key, api_endpoint=api_endpoint)

/opt/anaconda3/lib/python3.7/site-packages/typeguard/__init__.py:917: UserWarning: no type annotations present -- not typechecking kili.queries.lock.QueriesLock.count_locks
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


## 1 - Context

## 1.1 Agreement

Let's say you have a trained machine learning model $m$, which can, given a data $x$, output a prediction (ie, an inference label) $l^i = m(x)$.

What you will probably want to do is monitor the quality of such predictions, as the model evolves. Kili allows you to better monitor and iterate on your model, thanks to the concept of agreement. An agreement is a quantitative measure of similarity between two different labels. In Kili, there are three main features derived from agreement : 

- [Consensus](https://cloud.kili-technology.com/docs/quality/consensus/#docsNav), which is the agreement between two labelers.
- [Honeypot](https://cloud.kili-technology.com/docs/quality/honeypot/) which is the agreement between a "super human annotator" and a labeler.
- **Inference**, which is the agreement between a machine learning inference label and a human.

Those number can be monitored from the interface as the [quality KPIs](https://cloud.kili-technology.com/docs/quality/quality-kpis/). You can find how the agreement is computed [here](https://cloud.kili-technology.com/docs/quality/consensus/#consensus-calculation)

In this tutorial, we will put an emphasis on **Inference**.

## 1.2 Use cases 

We identify two main use cases for the use of **inference** :

1. **You have a model in production**. When it receives assets, it automatically feeds a Kili project with both the asset and the predicted label. **You also have human workforce, whose job is to monitor the quality of the model**. They regularly manually label some data seen by the model.
    - When a human submits a label, the inference score for that label is automatically computed using the predicted label.
    - Low inference scores can indicate either a model performing badly on some kind of data, or a disagreement between humans and the model. This can help you to :
    
        - `Detect data drift`
        - `Identify data on which the model needs improvement`
       
       
2. **You used Kili to label data**, and you have **the first iteration of your model**. You can use **a part of the dataset as testing data**, and quickly get **test scores**. You could of course use your own metrics (rather than our own definition of agreement), but using Kili allows you to quickly filter and indentify the assets where your model is most different from the ground truth.
    - When you push an inference label on an asset, the inference score is automatically computed for all most recent labels of the different people who labeled this asset.
    - You can filter on low inference score, to understand why your model is failing, and how to fix it (getting more data, splitting or merging categories, etc...)


Using Kili for monitoring or developing your model allows you to quickly iterate on the data used to train your model, allowing to get a better model faster.

# 2 - In practice

## 2.1 Use case 1

We start by creating a project and defining a model which, given an asset input x, returns a category (random in our example)

In [2]:
import random
random.seed(10)

json_interface ={
    "jobs": {
        "CLASSIFICATION_JOB": {
            "mlTask": "CLASSIFICATION",
            "content": {
                "categories": {
                    "RED": {"name": "Red"},
                    "BLACK": {"name": "Black"},
                    "WHITE": {"name": "White"},
                    "GREY": {"name": "Grey"}
                },
                "input": "radio"
            },
            "required": 0,
            "isChild": False,
            "instruction": "Color"
        }
    }
}

project_id = kili.create_project(
    title='Project demo inference',
    input_type='IMAGE',
    json_interface=json_interface
)['id']

categories = ['RED', 'BLACK', 'WHITE', 'GREY']


def machine_learning_model(x):
    return random.choice(categories)

Then we can simulate that our model is in production. Each time it receives an asset, it pushes it as well as the label it predicted to the project.

In [3]:
stream_of_assets = [
    {
        'url': "https://storage.googleapis.com/label-public-staging/recipes/inference/black_car.jpg",
        'external_id': 'black_car.jpg'
    },
    {
        'url': "https://storage.googleapis.com/label-public-staging/recipes/inference/grey_car.jpg",
        'external_id': 'grey_car.jpg'
    },
    {
        'url': "https://storage.googleapis.com/label-public-staging/recipes/inference/white_car.jpg",
        'external_id': 'white_car.jpg'
    },
    {
        'url': "https://storage.googleapis.com/label-public-staging/recipes/inference/red_car.jpg",
        'external_id': 'red_car.jpg'
    }
]

In [4]:
for i, asset in enumerate(stream_of_assets):
    kili.append_many_to_dataset(
        project_id=project_id,
        content_array=[asset['url']],
        external_id_array=[asset['external_id']]
    )
    asset_id = kili.assets(project_id=project_id, fields=['id'])[i]['id']
    predicted_category = machine_learning_model(asset)
    inference_label = {
        "CLASSIFICATION_JOB": {
            "categories": [{"name": predicted_category}]
        }
    }
    kili.append_to_labels(
        json_response=inference_label,
        label_asset_id=asset_id,
        label_type='INFERENCE',
    )

100%|██████████| 4/4 [00:00<00:00, 69.09it/s]


Then, human labelers can annotate a subsample of the assets pushed to Kili. 

Note : you can even automatically [prioritize assets](https://cloud.kili-technology.com/docs/python-graphql-api/recipes/import_assets/#setting-up-a-labeling-priority) to be reviewed by a human by using the model's uncertainty. When the model is unsure of its predictions, it may indicate wrong labels.

In [5]:
assets = kili.assets(project_id=project_id, fields=['id'])
ground_truths = ['BLACK', 'GREY', 'WHITE', 'RED']

for i, asset in enumerate(assets):
    human_label = {
        "CLASSIFICATION_JOB": {
            "categories": [{"name": ground_truths[i]}]
        }
    }
    kili.append_to_labels(
        json_response=human_label,
        label_asset_id=asset['id'],
        label_type='DEFAULT',
    )

100%|██████████| 4/4 [00:00<00:00, 47.98it/s]


You can now fetch the agreement between the human and the model, for human labels :

In [6]:
labels = kili.labels(project_id=project_id,
            fields=['inferenceMark', 'id', 'labelOf.id'], type_in=['DEFAULT'])
labels

[{'labelOf': {'id': 'ckpqjze9j00vsac9k897j2w73'},
  'inferenceMark': 0.5,
  'id': 'ckpqjzfml00xsac9k1i8scw4q'},
 {'labelOf': {'id': 'ckpqjzen000w9ac9kcxd8deo3'},
  'inferenceMark': 1,
  'id': 'ckpqjzfta00yaac9khdtcgxf6'},
 {'labelOf': {'id': 'ckpqjzeys00wqac9k4mdt2h7k'},
  'inferenceMark': 0.5,
  'id': 'ckpqjzg0b00ytac9k671ycpjw'},
 {'labelOf': {'id': 'ckpqjzf9e00x8ac9k14krfy0k'},
  'inferenceMark': 1,
  'id': 'ckpqjzg8x00zcac9k7bznhuyv'}]

In [7]:
# For testing
assert labels[0]['inferenceMark'] < 1
assert labels[1]['inferenceMark'] == 1

This allows you to identify problems :

In [8]:
for label in labels:
    if label['inferenceMark'] < 1:
        inference_label = kili.labels(asset_id=label['labelOf']['id'], type_in=['INFERENCE'])[0]
        human_label = kili.labels(label_id=label['id'])[0]
        inference_category = inference_label['jsonResponse']['CLASSIFICATION_JOB']['categories'][0]['name']
        human_category = human_label['jsonResponse']['CLASSIFICATION_JOB']['categories'][0]['name']
        print(f'The model predicted {inference_category} but the human predicted {human_category}')

The model predicted RED but the human predicted BLACK
The model predicted GREY but the human predicted WHITE


You can also find the assets with most disagreement directly from the interface with the filter "Human/Model IOU". Low IOU indicates low agreement : 

![inference](https://storage.googleapis.com/label-public-staging/recipes/inference/inference_filter.png)

## 2.2 Use case 2

We can invert the previous use case. We start by having a human labeled dataset, and we insert model predictions, to simulate testing our model on test data.

In [9]:
project_id = kili.create_project(
    title='Project demo inference 2',
    input_type='IMAGE',
    json_interface=json_interface
)['id']

In [10]:
labeled_assets = [
    {
        'url': "https://storage.googleapis.com/label-public-staging/recipes/inference/black_car.jpg",
        'external_id': 'black_car.jpg'
    },
    {
        'url': "https://storage.googleapis.com/label-public-staging/recipes/inference/grey_car.jpg",
        'external_id': 'grey_car.jpg'
    },
    {
        'url': "https://storage.googleapis.com/label-public-staging/recipes/inference/white_car.jpg",
        'external_id': 'white_car.jpg'
    },
    {
        'url': "https://storage.googleapis.com/label-public-staging/recipes/inference/red_car.jpg",
        'external_id': 'red_car.jpg'
    }
]
ground_truths = ['BLACK', 'GREY', 'WHITE', 'RED']

In [11]:
for i, (asset, category) in enumerate(zip(labeled_assets, ground_truths)):
    kili.append_many_to_dataset(
        project_id=project_id,
        content_array=[asset['url']],
        external_id_array=[asset['external_id']]
    )
    asset_id = kili.assets(project_id=project_id, fields=['id'])[i]['id']
    human_label = {
        "CLASSIFICATION_JOB": {
            "categories": [{"name": category}]
        }
    }
    kili.append_to_labels(
        json_response=human_label,
        label_asset_id=asset_id,
        label_type='DEFAULT',
    )

100%|██████████| 4/4 [00:00<00:00, 79.20it/s]


Then our model is fit using maybe 80% of the training data. We can then run it against the remaining 20%, and upload its predictions to Kili :

In [12]:
assets = kili.assets(project_id=project_id, fields=['id'])

for i, asset in enumerate(assets):
    test_label = {
        "CLASSIFICATION_JOB": {
            "categories": [{"name": machine_learning_model(asset)}]
        }
    }
    kili.append_to_labels(
        json_response=test_label,
        label_asset_id=asset['id'],
        label_type='INFERENCE',
    )

100%|██████████| 4/4 [00:00<00:00, 65.84it/s]


In [13]:
labels = kili.labels(project_id=project_id,
            fields=['inferenceMark', 'id', 'labelOf.id'], type_in=['DEFAULT'])
labels

[{'labelOf': {'id': 'ckpqjzgrp010aac9k3a8c6oy7'},
  'inferenceMark': 1,
  'id': 'ckpqjzgw2010hac9ked5609mq'},
 {'labelOf': {'id': 'ckpqjzh3k0110ac9k0j6r5eqw'},
  'inferenceMark': 1,
  'id': 'ckpqjzh7d0116ac9kgk9gbq8f'},
 {'labelOf': {'id': 'ckpqjzhek011pac9k6ppf3zf0'},
  'inferenceMark': 0.5,
  'id': 'ckpqjzhir011wac9k7y9jarir'},
 {'labelOf': {'id': 'ckpqjzhqj012gac9kgtbv58f4'},
  'inferenceMark': 0.5,
  'id': 'ckpqjzhue012mac9k7vof5432'}]

In [14]:
# For testing
assert labels[0]['inferenceMark'] == 1
assert labels[2]['inferenceMark'] < 1

In [15]:
for label in labels:
    if label['inferenceMark'] < 1:
        inference_label = kili.labels(asset_id=label['labelOf']['id'], type_in=['INFERENCE'])[0]
        human_label = kili.labels(label_id=label['id'])[0]
        inference_category = inference_label['jsonResponse']['CLASSIFICATION_JOB']['categories'][0]['name']
        human_category = human_label['jsonResponse']['CLASSIFICATION_JOB']['categories'][0]['name']
        print(f'The human predicted {human_category} but the model predicted {inference_category}')

The human predicted WHITE but the model predicted GREY
The human predicted RED but the model predicted WHITE


You can also find the assets where the prediction and the human disagree most directly from the interface with the filter "Human/Model IOU" : 

![inference](https://storage.googleapis.com/label-public-staging/recipes/inference/inference_test_filter.png)

In this tutorial, we accomplished the following:

We introduced the concept of Kili inference labels. We showed how to make use of such labels, in two practical use cases.

You can also visit the Kili website or Kili documentation for more info!